In [2]:
!pip install pyTelegramBotAPI

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install opencv-python

    100% |████████████████████████████████| 26.6MB 2.4MB/s eta 0:00:01
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!ls

girl.jpg  girl.jpg.1  Untitled.ipynb


In [4]:
!ls

girl.jpg  girl.jpg.1  test.jpg	Untitled.ipynb


In [4]:
!git clone https://github.com/PhotoLabDevelopment/hackathon-sdk.git

Cloning into 'hackathon-sdk'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 1), reused 17 (delta 1), pack-reused 0
Unpacking objects: 100% (17/17), done.
Checking connectivity... done.


In [11]:
!mv hackathon-sdk hackathon_sdk

In [12]:


python3 animefy.py ~/photohack/inputs ~/photohack/outputs ~/photohack/repos/stylegan-encoder ~/photohack/repos/TwinGAN ~/photohack/models/256



In [4]:
!ls photohack/inputs

nikulin_dmitry_1.jpg


In [8]:
!ls photohack/models/256

checkpoint  graph.pbtxt				   model.ckpt-100000.index
flags.txt   model.ckpt-100000.data-00000-of-00001  model.ckpt-100000.meta


In [5]:
!pwd

/home/ubuntu/workspace


In [13]:




content_filename = 'girl.jpg'
if not os.path.exists(content_filename):
    api.download_file('http://soft.photolab.me/samples/girl.jpg', content_filename)


content_url = api.image_upload(content_filename)
print('content_url: {}'.format(content_url))

template_name = api.template_upload(resourses_filename)
print('template_name: {}'.format(template_name))

content_url: http://temp-images.ws.pho.to/c3c36184e243c14de064ee2fb8d837fb1eddf89c.jpeg
template_name: 1001525


In [15]:
result_url = api.template_process('deepmask_process',[{
    'url' : content_url
}])
print('for template_name: {}, result_url: {}'.format(template_name, result_url))

for template_name: 1001525, result_url: http://worker-images.ws.pho.to/i2/1b181467821c40ac7e151b5270cde5f252291ba2_result.png


In [ ]:



main('/home/ubuntu/workspace/photohack/inputs',
    '/home/ubuntu/workspace/photohack/outputs',
    '/home/ubuntu/workspace/photohack/repos/stylegan-encoder',
    '/home/ubuntu/workspace/photohack/repos/TwinGAN',
    '/home/ubuntu/workspace/photohack/models/256')

In [1]:
import imageio

In [ ]:
import argparse
from pathlib import Path
import subprocess
import tempfile
import telebot
import requests
import cv2
import os.path
import imageio
import numpy as np
from hackathon_sdk.python.client_photolab import ClientPhotolab

def main(input_dir: Path, output_dir: Path, stylegan_dir: Path, twingan_dir: Path, twingan_model_dir: Path):
    with tempfile.TemporaryDirectory(prefix='animefy-') as work_dir:
        work_dir = Path(work_dir)
        assert work_dir.exists() and work_dir.is_dir()
    
        aligned_images_dir = work_dir / 'aligned'

        subprocess.check_call([
            'python3',
            'align_images.py',
            '--output_size', '256',
            str(input_dir),
            str(aligned_images_dir),
        ], cwd=str(stylegan_dir))

        anime_face_dir = work_dir / 'anime_face'

        subprocess.check_call([
            'python3',
            'inference/image_translation_infer.py',
            '--model_path', str(twingan_model_dir),
            '--image_hw', '256',
            '--input_tensor_name', 'sources_ph',
            '--output_tensor_name', 'custom_generated_t_style_source:0',
            '--input_image_path', str(aligned_images_dir),
            '--output_image_path', str(anime_face_dir),
        ], cwd=str(twingan_dir))

        faces_files = sorted(aligned_images_dir.iterdir())
        anime_files = sorted(anime_face_dir.iterdir())

        for (face_path, anime_path) in zip(faces_files, anime_files):
            face = imageio.imread(face_path)
            anime = imageio.imread(anime_path)

            assert face_path.name == anime_path.name
            assert face.shape == (256, 256, 3)
            assert anime.shape == (256, 256, 3)

            output_path = output_dir / face_path.name
            output = np.hstack([face, anime])
            imageio.imwrite(output_path, output)

images_path_input = '/home/ubuntu/workspace/photohack/inputs/'
images_path_output = '/home/ubuntu/workspace/photohack/outputs/'

resourses_filename = 'resources.zip'
if not os.path.exists(resourses_filename):
    api.download_file('http://soft.photolab.me/samples/resources.zip', resourses_filename)
api = ClientPhotolab()
bot = telebot.TeleBot("655293955:AAG9YBDrgh-mX5dAy6HIAsoP_W2Y7tHsWzs")

@bot.message_handler(content_types=['document', 'photo'])
def handle_docs_audio(message):
    chat_id = message.chat.id
    photo_id = message.photo[len(message.photo) - 1].file_id
    
    file_info = bot.get_file(photo_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open(images_path_input + 'test.jpg', 'wb') as new_file:
        new_file.write(downloaded_file)
    content_url = api.image_upload(images_path_input + 'test.jpg')
    result_url = api.template_process('deepmask_process',[{
        'url' : content_url
    }])
    print('done')
    
    bot.reply_to(message, result_url)
    _result_url = api.template_process(2698,[{
        'url' : content_url
    }])
    bot.reply_to(message, _result_url)
    
    bot.reply_to(message, result_url)
    result_url = api.template_process(1623,[{
        'url' : content_url
    }])
    bot.reply_to(message, result_url)
    
    result_url = api.template_process(682,[{
        'url' : result_url
    }])
    bot.reply_to(message, result_url)
    
    result_url = api.template_process(1339,[{
        'url' : result_url
    }])
    bot.reply_to(message, result_url)
    
    print('Begin deeplearning stuff')
    main(
    Path(images_path_input),
    Path(images_path_output),
    Path('/home/ubuntu/workspace/photohack/repos/stylegan-encoder'),
    Path('/home/ubuntu/workspace/photohack/repos/TwinGAN'),
    Path('/home/ubuntu/workspace/photohack/models/256'))
    print('End deeplearning stuff')
    content_url = api.image_upload(images_path_output + 'test_01.png')
    bot.reply_to(message, content_url)
    #
    
    
#     photo = open('test.jpg', 'rb')
#     bot.send_photo(chat_id, photo)
    
    #bot.reply_to(message, message.photo.file_id)
    #message.photo.file_id
    #bot.reply_to(message, message.chat)

@bot.message_handler(func=lambda m: True)
def echo_all(message):
    chat_id = message.chat.id
    print(message)
#     photo = open('girl.jpg', 'rb')
#     bot.send_photo(chat_id, photo)
    bot.reply_to(message, message.chat)

while 1:
    try:
        bot.polling()
    except Exception as e:
        print(e)
        pass

Exception in thread PollingThread:
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/util.py", line 59, in run
    task(*args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/__init__.py", line 276, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1), timeout=timeout)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/__init__.py", line 246, in get_updates
    json_updates = apihelper.get_updates(self.token, offset, limit, timeout, allowed_updates)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/apihelper.py", line 180, in get_updates
    return _make_request(token, method_url, params=payload)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/apihelper.py", line 56, in _make_request
    return _check_result(method_name, result)['result']
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/apihelper.py", line 75, in _check_re

Exception in thread PollingThread:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/urllib3/connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/urllib3/util/connection.py", line 56, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
OSError: [Errno 24] Too many open files

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/pytho

2019-06-23 14:11:06,808 - util.py:65 - ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 409 Conflict. Response body:\n[b\'{"ok":false,"error_code":409,"description":"Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"}\']',)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/util.py", line 59, in run
    task(*args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/__init__.py", line 276, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1), timeout=timeout)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/__init__.py", line 246, in get_updates
    json_updates = apihelper.get_updates(self.token, offset, limit, timeout, allowed_updates)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/telebot/apihelper.py", line 180, in get_updates
    return _make_request(token

done
Begin deeplearning stuff
End deeplearning stuff
done
Begin deeplearning stuff
End deeplearning stuff
done
Begin deeplearning stuff
End deeplearning stuff


In [3]:
import sys
import os
sys.path.append('./')

import numpy as np
import tensorflow as tf


import util_io

ModuleNotFoundError: No module named 'util_io'

In [12]:
!pip3 install util_io

  Could not find a version that satisfies the requirement util_io (from versions: )
No matching distribution found for util_io
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import os
sys.path.append('photohack/repos/TwinGAN')

In [16]:
from photohack.repos.TwinGAN.inference.image_translation_infer import ImageInferer 

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/absl/flags/_validators.py:359: UserWarning: Flag --model_path has a non-None default value; therefore, mark_flag_as_required will pass even if flag is not specified in the command line!
  'command line!' % flag_name)


In [ ]:
import argparse
import subprocess
import tempfile
from pathlib import Path
import telebot
import requests
import cv2
import os.path
from hackathon_sdk.python.client_photolab import ClientPhotolab

import sys
import os
sys.path.append('./')

import numpy as np
import tensorflow as tf


import util_io
tf.flags.DEFINE_string("model_path", "", "Path containing a checkpoint.")
tf.flags.DEFINE_integer('image_hw', 256, 'height and width of the input image.')
tf.flags.DEFINE_string('input_tensor_name', None, 'Optional input tensor name. e.g. sources_ph.')
tf.flags.DEFINE_string('output_tensor_name', None, 'e.g. custom_generated_t_style_source:0')

tf.flags.mark_flags_as_required(['model_path', 'output_tensor_name'])
FLAGS = tf.flags.FLAGS

def main(input_dir: Path, output_dir: Path, stylegan_dir: Path, twingan_dir: Path, twingan_model_dir: Path):
    with tempfile.TemporaryDirectory(prefix='animefy-') as work_dir:
        work_dir = Path(work_dir)
        assert work_dir.exists() and work_dir.is_dir()

        aligned_images_dir = work_dir / 'aligned'

        subprocess.check_call([
            'python3',
            'align_images.py',
            '--output_size', '256',
            str(input_dir),
            str(aligned_images_dir),
        ], cwd=str(stylegan_dir))

        subprocess.check_call([
            'python3',
            'inference/image_translation_infer.py',
            '--model_path', str(twingan_model_dir),
            '--image_hw', '256',
            '--input_tensor_name', 'sources_ph',
            '--output_tensor_name', 'custom_generated_t_style_source:0',
            '--input_image_path', str(aligned_images_dir),
            '--output_image_path', str(output_dir),
        ], cwd=str(twingan_dir))

images_path_input = '/home/ubuntu/workspace/photohack/inputs/'
images_path_output = '/home/ubuntu/workspace/photohack/outputs/'

resourses_filename = 'resources.zip'
if not os.path.exists(resourses_filename):
    api.download_file('http://soft.photolab.me/samples/resources.zip', resourses_filename)
api = ClientPhotolab()
bot = telebot.TeleBot("655293955:AAG9YBDrgh-mX5dAy6HIAsoP_W2Y7tHsWzs")

@bot.message_handler(content_types=['document', 'photo'])
def handle_docs_audio(message):
    chat_id = message.chat.id
    photo_id = message.photo[len(message.photo) - 1].file_id
    
    file_info = bot.get_file(photo_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open(images_path_input + 'test.jpg', 'wb') as new_file:
        new_file.write(downloaded_file)
    content_url = api.image_upload(images_path_input + 'test.jpg')
    result_url = api.template_process('deepmask_process',[{
        'url' : content_url
    }])
    print('done')
    
    bot.reply_to(message, result_url)
    _result_url = api.template_process(2708,[{
        'url' : content_url
    }])
    bot.reply_to(message, _result_url)
    
    bot.reply_to(message, result_url)
    result_url = api.template_process(1623,[{
        'url' : content_url
    }])
    bot.reply_to(message, result_url)
    
    result_url = api.template_process(682,[{
        'url' : result_url
    }])
    bot.reply_to(message, result_url)
    
    result_url = api.template_process(1339,[{
        'url' : result_url
    }])
    bot.reply_to(message, result_url)
    
    print('Begin deeplearning staff')
    main(images_path_input,
    images_path_output,
    '/home/ubuntu/workspace/photohack/repos/stylegan-encoder',
    '/home/ubuntu/workspace/photohack/repos/TwinGAN',
    '/home/ubuntu/workspace/photohack/models/256')
    print('End deeplearning staff')
    content_url = api.image_upload(images_path_output + 'test_01.png')
    bot.reply_to(message, content_url)
    #
    
    
#     photo = open('test.jpg', 'rb')
#     bot.send_photo(chat_id, photo)
    
    #bot.reply_to(message, message.photo.file_id)
    #message.photo.file_id
    #bot.reply_to(message, message.chat)

@bot.message_handler(func=lambda m: True)
def echo_all(message):
    chat_id = message.chat.id
    print(message)
#     photo = open('girl.jpg', 'rb')
#     bot.send_photo(chat_id, photo)
    bot.reply_to(message, message.chat)

while 1:
    try:
        bot.polling()
    except Exception as e:
        print(e)
        pass

done
Begin deeplearning staff
End deeplearning staff


In [2]:
!ls /home/ubuntu/workspace/photohack/outputs/

nikulin_dmitry_1_01.png  test_01.png
